# SDG Multi-Label Classification: EDA & Visualization

## Project Overview
This notebook implements a comprehensive **multi-label classification system** for predicting Sustainable Development Goals (SDGs) from scientific publication data using a **hybrid approach** combining clustering and classification techniques.

### Dataset Description
- **Total entries**: 7,147 scientific publications from Google Scholar
- **Input features**: Title (text), Authors, Year, Cited count
- **Target labels**: Multi-label SDG categories (e.g., "SDG 3; SDG 9; SDG 13")

### Methodology
1. **Data Preprocessing**: Handle missing values, text cleaning, feature normalization
2. **Feature Engineering**: TF-IDF/BERT text vectorization + numerical features
3. **Clustering Analysis**: K-Means/DBSCAN for semantic grouping
4. **Hybrid Classification**: One-vs-Rest with ensemble methods + clustering features
5. **Evaluation**: Multi-label metrics (Exact Match, Hamming Loss, F1-Score, Jaccard)

---

## 1. Import Libraries & Load Dataset

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import re
from collections import Counter

# Machine Learning libraries
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    hamming_loss, jaccard_score, classification_report,
    multilabel_confusion_matrix, silhouette_score
)

# Text processing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from wordcloud import WordCloud

# Plotting and visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# Model persistence
import joblib

# Suppress warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Download NLTK data (run once)
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('punkt')
    nltk.download('stopwords')

# Configure matplotlib
plt.style.use('default')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

In [ ]:
# Load the dataset
DATA_PATH = "../data/2503_to_3336_preprocessing_labeling.csv"

try:
    df = pd.read_csv(DATA_PATH)
    print("✅ Dataset loaded successfully!")
    print(f"Dataset shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
except FileNotFoundError:
    print(f"❌ Error: File not found at {DATA_PATH}")
    print("Please make sure the data file exists in the correct location.")

# Display basic information about the dataset
print("\n" + "="*60)
print("DATASET OVERVIEW")
print("="*60)

print(f"\n📊 Dataset Dimensions:")
print(f"   Rows: {df.shape[0]:,}")
print(f"   Columns: {df.shape[1]}")

print(f"\n📋 Column Information:")
print(df.info())

print(f"\n🔍 First 5 rows:")
df.head()

In [ ]:
# Check for missing values
print("🔍 Missing Values Analysis:")
print("="*40)
missing_values = df.isnull().sum()
missing_percentages = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Count': missing_values.values,
    'Missing Percentage': missing_percentages.values
})

print(missing_df)

# Basic statistics for numerical columns
print("\n📈 Numerical Columns Statistics:")
print("="*40)
print(df.describe())

# Check data types
print("\n🏷️ Data Types:")
print("="*40)
print(df.dtypes)

# Sample of the SDGs column to understand the format
print("\n🎯 Sample SDG Labels:")
print("="*40)
for i, sdg in enumerate(df['SDGs'].head(10)):
    print(f"{i+1:2d}. {sdg}")

# Check unique values in categorical columns
print(f"\n📊 Unique Years: {df['Year'].nunique()} (Range: {df['Year'].min()} - {df['Year'].max()})")
print(f"📊 Unique Citation counts: {df['Cited'].nunique()} (Range: {df['Cited'].min()} - {df['Cited'].max()})")

## 2. Data Preprocessing

### Handle Missing Values & Data Cleaning

In [ ]:
# Create a copy for preprocessing
df_processed = df.copy()

print("🛠️ PREPROCESSING STEPS")
print("="*50)

# 1. Handle missing values
print("\n1️⃣ Handling Missing Values...")

# Fill missing Authors with 'Unknown'
df_processed['Authors'] = df_processed['Authors'].fillna('Unknown')

# Fill missing Title with empty string
df_processed['Title'] = df_processed['Title'].fillna('')

# Fill missing Year with median
median_year = df_processed['Year'].median()
df_processed['Year'] = df_processed['Year'].fillna(median_year)

# Fill missing Cited with 0
df_processed['Cited'] = df_processed['Cited'].fillna(0)

print(f"   ✅ Authors: {df['Authors'].isnull().sum()} → {df_processed['Authors'].isnull().sum()}")
print(f"   ✅ Title: {df['Title'].isnull().sum()} → {df_processed['Title'].isnull().sum()}")
print(f"   ✅ Year: {df['Year'].isnull().sum()} → {df_processed['Year'].isnull().sum()}")
print(f"   ✅ Cited: {df['Cited'].isnull().sum()} → {df_processed['Cited'].isnull().sum()}")

# 2. Parse SDG labels
print("\n2️⃣ Parsing SDG Labels...")

def parse_sdgs(sdg_string):
    """Parse SDG labels from string format to list"""
    if pd.isna(sdg_string):
        return []
    
    # Extract SDG numbers using regex
    sdg_matches = re.findall(r'SDG (\d+)', str(sdg_string))
    return [f"SDG_{num}" for num in sdg_matches]

# Apply SDG parsing
df_processed['SDG_labels'] = df_processed['SDGs'].apply(parse_sdgs)

# Remove rows with no SDG labels
initial_length = len(df_processed)
df_processed = df_processed[df_processed['SDG_labels'].apply(len) > 0]
final_length = len(df_processed)

print(f"   ✅ Rows with valid SDG labels: {final_length:,} (removed {initial_length - final_length:,} rows)")

# 3. Clean and preprocess title text
print("\n3️⃣ Text Preprocessing...")

def clean_text(text):
    """Clean and preprocess text data"""
    if pd.isna(text):
        return ""
    
    # Convert to lowercase
    text = str(text).lower()
    
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove extra whitespaces
    text = ' '.join(text.split())
    
    return text

df_processed['Title_cleaned'] = df_processed['Title'].apply(clean_text)

print(f"   ✅ Text cleaning completed")
print(f"   ✅ Average title length: {df_processed['Title_cleaned'].str.len().mean():.1f} characters")

print(f"\n📊 Final processed dataset shape: {df_processed.shape}")

# Display processed sample
print("\n🔍 Sample of processed data:")
df_processed[['Title_cleaned', 'Year', 'Cited', 'SDG_labels']].head()

In [ ]:
# SDG Distribution Analysis
print("🎯 SDG DISTRIBUTION ANALYSIS")
print("="*50)

# Extract all SDG labels
all_sdgs = []
for sdg_list in df_processed['SDG_labels']:
    all_sdgs.extend(sdg_list)

sdg_counts = pd.Series(all_sdgs).value_counts()
print(f"\n📈 Total SDG instances: {len(all_sdgs)}")
print(f"📈 Unique SDG classes: {len(sdg_counts)}")
print(f"📈 Average SDGs per publication: {len(all_sdgs) / len(df_processed):.2f}")

print(f"\n🏆 Top 10 Most Common SDGs:")
print(sdg_counts.head(10))

# Multi-label statistics
sdg_per_doc = df_processed['SDG_labels'].apply(len)
print(f"\n📊 Multi-label Statistics:")
print(f"   Single label: {sum(sdg_per_doc == 1):,} ({sum(sdg_per_doc == 1)/len(df_processed)*100:.1f}%)")
print(f"   Multiple labels: {sum(sdg_per_doc > 1):,} ({sum(sdg_per_doc > 1)/len(df_processed)*100:.1f}%)")
print(f"   Max labels per doc: {sdg_per_doc.max()}")
print(f"   Average labels per doc: {sdg_per_doc.mean():.2f}")

# Visualize SDG distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. SDG frequency bar plot
axes[0, 0].bar(range(len(sdg_counts)), sdg_counts.values)
axes[0, 0].set_title('SDG Label Frequency Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('SDG Classes')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Labels per document histogram
axes[0, 1].hist(sdg_per_doc, bins=range(1, sdg_per_doc.max() + 2), alpha=0.7, edgecolor='black')
axes[0, 1].set_title('Distribution of Labels per Document', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Number of SDG Labels')
axes[0, 1].set_ylabel('Number of Documents')

# 3. Year distribution
axes[1, 0].hist(df_processed['Year'], bins=20, alpha=0.7, edgecolor='black')
axes[1, 0].set_title('Publication Year Distribution', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Year')
axes[1, 0].set_ylabel('Number of Publications')

# 4. Citation distribution (log scale)
cited_nonzero = df_processed[df_processed['Cited'] > 0]['Cited']
axes[1, 1].hist(np.log1p(cited_nonzero), bins=30, alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Citation Distribution (Log Scale)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Log(Citations + 1)')
axes[1, 1].set_ylabel('Number of Publications')

plt.tight_layout()
plt.show()

# Top SDGs pie chart
plt.figure(figsize=(10, 8))
top_10_sdgs = sdg_counts.head(10)
plt.pie(top_10_sdgs.values, labels=top_10_sdgs.index, autopct='%1.1f%%', startangle=90)
plt.title('Top 10 SDG Distribution', fontsize=16, fontweight='bold')
plt.axis('equal')
plt.show()

## 3. Text Vectorization (TF-IDF & Word Analysis)

### TF-IDF Vectorization and Text Feature Analysis

In [ ]:
# Text Analysis and Vectorization
print("📝 TEXT ANALYSIS & VECTORIZATION")
print("="*50)

# 1. Basic text statistics
print("\n1️⃣ Basic Text Statistics:")
title_lengths = df_processed['Title_cleaned'].str.len()
title_words = df_processed['Title_cleaned'].str.split().str.len()

print(f"   Average title length: {title_lengths.mean():.1f} characters")
print(f"   Average words per title: {title_words.mean():.1f}")
print(f"   Max title length: {title_lengths.max()} characters")
print(f"   Max words per title: {title_words.max()}")

# 2. Word frequency analysis
print("\n2️⃣ Word Frequency Analysis:")
all_words = ' '.join(df_processed['Title_cleaned']).split()
word_freq = Counter(all_words)

print(f"   Total words: {len(all_words):,}")
print(f"   Unique words: {len(word_freq):,}")
print(f"   Top 10 most common words: {word_freq.most_common(10)}")

# 3. TF-IDF Vectorization
print("\n3️⃣ TF-IDF Vectorization:")

# Initialize TF-IDF vectorizer
tfidf = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2),  # Unigrams and bigrams
    min_df=2,  # Ignore terms that appear in less than 2 documents
    max_df=0.95  # Ignore terms that appear in more than 95% of documents
)

# Fit and transform the text data
tfidf_matrix = tfidf.fit_transform(df_processed['Title_cleaned'])

print(f"   ✅ TF-IDF Matrix Shape: {tfidf_matrix.shape}")
print(f"   ✅ Vocabulary Size: {len(tfidf.vocabulary_)}")
print(f"   ✅ Feature Names (first 10): {tfidf.get_feature_names_out()[:10]}")

# 4. Most important TF-IDF features
print("\n4️⃣ Top TF-IDF Features:")
feature_names = tfidf.get_feature_names_out()
tfidf_scores = tfidf_matrix.mean(axis=0).A1
feature_importance = sorted(zip(feature_names, tfidf_scores), key=lambda x: x[1], reverse=True)

print("   Top 20 TF-IDF features:")
for i, (feature, score) in enumerate(feature_importance[:20]):
    print(f"   {i+1:2d}. {feature:20s} → {score:.4f}")

# Visualize text statistics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Title length distribution
axes[0, 0].hist(title_lengths, bins=30, alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Title Length Distribution (Characters)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Number of Characters')
axes[0, 0].set_ylabel('Frequency')

# 2. Words per title distribution
axes[0, 1].hist(title_words, bins=20, alpha=0.7, edgecolor='black')
axes[0, 1].set_title('Words per Title Distribution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Number of Words')
axes[0, 1].set_ylabel('Frequency')

# 3. Top 20 words frequency
top_words = dict(word_freq.most_common(20))
axes[1, 0].bar(range(len(top_words)), list(top_words.values()))
axes[1, 0].set_title('Top 20 Most Frequent Words', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Words')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_xticks(range(len(top_words)))
axes[1, 0].set_xticklabels(list(top_words.keys()), rotation=45, ha='right')

# 4. TF-IDF feature importance
top_tfidf = dict(feature_importance[:20])
axes[1, 1].bar(range(len(top_tfidf)), [score for _, score in feature_importance[:20]])
axes[1, 1].set_title('Top 20 TF-IDF Features', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Features')
axes[1, 1].set_ylabel('Average TF-IDF Score')
axes[1, 1].set_xticks(range(len(top_tfidf)))
axes[1, 1].set_xticklabels([feat for feat, _ in feature_importance[:20]], rotation=45, ha='right')

plt.tight_layout()
plt.show()

# Create word cloud
print("\n5️⃣ Generating Word Cloud...")
try:
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(df_processed['Title_cleaned']))
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Word Cloud of Publication Titles', fontsize=16, fontweight='bold')
    plt.show()
    print("   ✅ Word cloud generated successfully!")
except Exception as e:
    print(f"   ❌ Error generating word cloud: {e}")

## 4. Feature Engineering & Combination

### Normalize Numerical Features and Combine with Text Features

In [ ]:
# Feature Engineering and Combination
print("⚙️ FEATURE ENGINEERING & COMBINATION")
print("="*50)

# 1. Normalize numerical features
print("\n1️⃣ Normalizing Numerical Features:")

# Extract numerical features
numerical_features = df_processed[['Year', 'Cited']].copy()

print(f"   Original Year range: {numerical_features['Year'].min()} - {numerical_features['Year'].max()}")
print(f"   Original Cited range: {numerical_features['Cited'].min()} - {numerical_features['Cited'].max()}")

# Initialize scaler
scaler = StandardScaler()

# Fit and transform numerical features
numerical_features_scaled = scaler.fit_transform(numerical_features)

print(f"   ✅ Numerical features normalized")
print(f"   Scaled Year mean: {numerical_features_scaled[:, 0].mean():.4f}, std: {numerical_features_scaled[:, 0].std():.4f}")
print(f"   Scaled Cited mean: {numerical_features_scaled[:, 1].mean():.4f}, std: {numerical_features_scaled[:, 1].std():.4f}")

# 2. Combine features
print("\n2️⃣ Combining Text and Numerical Features:")

from scipy.sparse import hstack, csr_matrix

# Combine TF-IDF features with normalized numerical features
combined_features = hstack([tfidf_matrix, csr_matrix(numerical_features_scaled)])

print(f"   TF-IDF matrix shape: {tfidf_matrix.shape}")
print(f"   Numerical features shape: {numerical_features_scaled.shape}")
print(f"   ✅ Combined features shape: {combined_features.shape}")

# 3. Create target matrix (multi-label binarization)
print("\n3️⃣ Creating Multi-label Target Matrix:")

mlb = MultiLabelBinarizer()
y_multilabel = mlb.fit_transform(df_processed['SDG_labels'])

print(f"   ✅ Target matrix shape: {y_multilabel.shape}")
print(f"   ✅ Number of unique SDG classes: {len(mlb.classes_)}")
print(f"   ✅ SDG classes: {mlb.classes_}")

# Display label distribution
label_sums = y_multilabel.sum(axis=0)
label_df = pd.DataFrame({
    'SDG': mlb.classes_,
    'Count': label_sums,
    'Percentage': (label_sums / len(df_processed)) * 100
}).sort_values('Count', ascending=False)

print(f"\n📊 Label Distribution:")
print(label_df)

# Visualize feature combination and target distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Numerical features before normalization
axes[0, 0].scatter(numerical_features['Year'], numerical_features['Cited'], alpha=0.6)
axes[0, 0].set_title('Numerical Features (Original)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Citations')

# 2. Numerical features after normalization
axes[0, 1].scatter(numerical_features_scaled[:, 0], numerical_features_scaled[:, 1], alpha=0.6)
axes[0, 1].set_title('Numerical Features (Normalized)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Year (Normalized)')
axes[0, 1].set_ylabel('Citations (Normalized)')

# 3. Target label distribution
axes[1, 0].bar(range(len(label_sums)), label_sums)
axes[1, 0].set_title('SDG Label Distribution', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('SDG Classes')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].tick_params(axis='x', rotation=45)

# 4. Labels per document distribution
labels_per_doc = y_multilabel.sum(axis=1)
axes[1, 1].hist(labels_per_doc, bins=range(1, labels_per_doc.max() + 2), alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Labels per Document Distribution', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Number of Labels')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# Feature summary
print(f"\n📋 FEATURE SUMMARY:")
print(f"   Text features (TF-IDF): {tfidf_matrix.shape[1]:,}")
print(f"   Numerical features: {numerical_features_scaled.shape[1]}")
print(f"   Total combined features: {combined_features.shape[1]:,}")
print(f"   Total samples: {combined_features.shape[0]:,}")
print(f"   Target classes: {y_multilabel.shape[1]}")

# Memory usage
print(f"\n💾 Memory Usage:")
print(f"   Combined features matrix: {combined_features.data.nbytes / 1024**2:.1f} MB (sparse)")
print(f"   Target matrix: {y_multilabel.nbytes / 1024**2:.1f} MB")

## 5. Clustering-Based Preprocessing

### K-Means and DBSCAN Analysis for Semantic Grouping

In [ ]:
# Clustering Analysis
print("🔍 CLUSTERING ANALYSIS")
print("="*50)

# 1. Find optimal number of clusters using elbow method and silhouette score
print("\n1️⃣ Finding Optimal Number of Clusters:")

max_clusters = 20
k_range = range(2, max_clusters + 1)
inertias = []
silhouette_scores = []

print("   Computing K-means for different cluster numbers...")

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(combined_features)
    
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(combined_features, cluster_labels))
    
    if k % 5 == 0:
        print(f"   ✅ K={k}: Inertia={kmeans.inertia_:.2f}, Silhouette={silhouette_scores[-1]:.4f}")

# Find optimal K using silhouette score
best_k_silhouette = k_range[np.argmax(silhouette_scores)]
best_silhouette_score = max(silhouette_scores)

print(f"\n   🏆 Best K (Silhouette): {best_k_silhouette} (Score: {best_silhouette_score:.4f})")

# Plot elbow curve and silhouette scores
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Elbow curve
axes[0].plot(k_range, inertias, 'bo-')
axes[0].set_title('Elbow Method for Optimal K', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Number of Clusters (K)')
axes[0].set_ylabel('Inertia')
axes[0].grid(True, alpha=0.3)

# Silhouette scores
axes[1].plot(k_range, silhouette_scores, 'ro-')
axes[1].axvline(x=best_k_silhouette, color='red', linestyle='--', alpha=0.7, label=f'Best K = {best_k_silhouette}')
axes[1].set_title('Silhouette Score vs Number of Clusters', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Number of Clusters (K)')
axes[1].set_ylabel('Silhouette Score')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 2. Perform K-means clustering with optimal K
print(f"\n2️⃣ Performing K-means Clustering (K={best_k_silhouette}):")

kmeans_final = KMeans(n_clusters=best_k_silhouette, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(combined_features)

print(f"   ✅ Clustering completed")
print(f"   Silhouette Score: {silhouette_score(combined_features, cluster_labels):.4f}")

# Cluster statistics
unique_clusters, cluster_counts = np.unique(cluster_labels, return_counts=True)
print(f"   Number of clusters: {len(unique_clusters)}")
print(f"   Cluster sizes: {dict(zip(unique_clusters, cluster_counts))}")

# 3. Analyze SDG distribution across clusters
print(f"\n3️⃣ Analyzing SDG Distribution Across Clusters:")

cluster_sdg_analysis = {}
for cluster_id in unique_clusters:
    cluster_mask = cluster_labels == cluster_id
    cluster_sdgs = y_multilabel[cluster_mask]
    
    # Calculate SDG frequency in this cluster
    sdg_freq = np.mean(cluster_sdgs, axis=0)
    cluster_sdg_analysis[cluster_id] = dict(zip(mlb.classes_, sdg_freq))
    
    print(f"   Cluster {cluster_id}: {cluster_counts[cluster_id]} documents")
    # Show top 3 SDGs in this cluster
    top_sdgs = sorted(zip(mlb.classes_, sdg_freq), key=lambda x: x[1], reverse=True)[:3]
    print(f"      Top SDGs: {', '.join([f'{sdg}({freq:.2f})' for sdg, freq in top_sdgs])}")

# 4. Visualize clustering results
print(f"\n4️⃣ Visualizing Clustering Results:")

# Use PCA for dimensionality reduction for visualization
pca = PCA(n_components=2, random_state=42)
features_2d = pca.fit_transform(combined_features.toarray())

print(f"   PCA explained variance ratio: {pca.explained_variance_ratio_}")
print(f"   Total explained variance: {sum(pca.explained_variance_ratio_):.4f}")

# Create cluster visualization
plt.figure(figsize=(14, 10))

# Plot clusters
colors = plt.cm.Set3(np.linspace(0, 1, len(unique_clusters)))
for i, cluster_id in enumerate(unique_clusters):
    mask = cluster_labels == cluster_id
    plt.scatter(features_2d[mask, 0], features_2d[mask, 1], 
               c=[colors[i]], label=f'Cluster {cluster_id} ({cluster_counts[i]})', 
               alpha=0.6, s=30)

plt.title('Document Clustering Visualization (PCA)', fontsize=16, fontweight='bold')
plt.xlabel(f'First Principal Component (Explained Variance: {pca.explained_variance_ratio_[0]:.3f})')
plt.ylabel(f'Second Principal Component (Explained Variance: {pca.explained_variance_ratio_[1]:.3f})')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 5. Create SDG-cluster heatmap
print(f"\n5️⃣ Creating SDG-Cluster Distribution Heatmap:")

# Prepare data for heatmap
cluster_sdg_matrix = []
for cluster_id in unique_clusters:
    cluster_mask = cluster_labels == cluster_id
    cluster_sdgs = y_multilabel[cluster_mask]
    sdg_freq = np.mean(cluster_sdgs, axis=0)
    cluster_sdg_matrix.append(sdg_freq)

cluster_sdg_matrix = np.array(cluster_sdg_matrix)

plt.figure(figsize=(16, 8))
sns.heatmap(cluster_sdg_matrix, 
           xticklabels=mlb.classes_,
           yticklabels=[f'Cluster {c}' for c in unique_clusters],
           annot=True, fmt='.2f', cmap='YlOrRd', cbar_kws={'label': 'SDG Frequency'})
plt.title('SDG Distribution Across Clusters', fontsize=16, fontweight='bold')
plt.xlabel('SDG Categories')
plt.ylabel('Clusters')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# 6. Add cluster features to the feature matrix
print(f"\n6️⃣ Adding Cluster Features:")

# Add cluster labels as additional features
cluster_features = cluster_labels.reshape(-1, 1)
enhanced_features = hstack([combined_features, csr_matrix(cluster_features)])

print(f"   Original features shape: {combined_features.shape}")
print(f"   Enhanced features shape: {enhanced_features.shape}")
print(f"   ✅ Cluster features added successfully")

## 6. Train-Test Split & Multi-label Preparation

### Prepare Data for Model Training

In [ ]:
# Train-Test Split
print("📊 TRAIN-TEST SPLIT PREPARATION")
print("="*50)

# 1. Prepare features and targets
print("\n1️⃣ Preparing Data for Split:")

X = combined_features  # Original features (without cluster)
X_enhanced = enhanced_features  # Features with cluster information
y = y_multilabel

print(f"   Original features (X): {X.shape}")
print(f"   Enhanced features (X_enhanced): {X_enhanced.shape}")
print(f"   Target labels (y): {y.shape}")

# 2. Perform stratified split for multi-label data
print("\n2️⃣ Performing Stratified Train-Test Split:")

# For multi-label data, we use a simple split as stratification is complex
test_size = 0.2
random_state = 42

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state
)

# Split enhanced features the same way
X_enhanced_train, X_enhanced_test, _, _ = train_test_split(
    X_enhanced, y, test_size=test_size, random_state=random_state
)

# Further split training data for validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=random_state
)

X_enhanced_train, X_enhanced_val, _, _ = train_test_split(
    X_enhanced_train, y_train, test_size=0.2, random_state=random_state
)

print(f"   ✅ Training set: {X_train.shape}")
print(f"   ✅ Validation set: {X_val.shape}")
print(f"   ✅ Test set: {X_test.shape}")

# 3. Extract cluster features separately
print("\n3️⃣ Extracting Cluster Features:")

# Get cluster features for each split
train_indices = np.arange(len(X))[:int(len(X) * 0.8)]  # First 80% for train+val
test_indices = np.arange(len(X))[int(len(X) * 0.8):]   # Last 20% for test

# Split cluster labels
cluster_train_val = cluster_labels[train_indices]
cluster_test = cluster_labels[test_indices]

# Further split train and validation
train_val_split = int(len(cluster_train_val) * 0.8)
cluster_train = cluster_train_val[:train_val_split]
cluster_val = cluster_train_val[train_val_split:]

print(f"   Cluster features - Train: {cluster_train.shape}")
print(f"   Cluster features - Validation: {cluster_val.shape}")
print(f"   Cluster features - Test: {cluster_test.shape}")

# 4. Analyze label distribution across splits
print("\n4️⃣ Analyzing Label Distribution Across Splits:")

def analyze_split_distribution(y_split, split_name):
    \"\"\"Analyze label distribution in a data split\"\"\"\n    label_sums = y_split.sum(axis=0)\n    total_samples = len(y_split)\n    \n    print(f\"   {split_name}:\")\n    print(f\"     Samples: {total_samples:,}\")\n    print(f\"     Total labels: {label_sums.sum():,}\")\n    print(f\"     Avg labels per sample: {label_sums.sum() / total_samples:.2f}\")\n    print(f\"     Most common SDG: {mlb.classes_[np.argmax(label_sums)]} ({label_sums.max()} instances)\")\n    return label_sums

train_dist = analyze_split_distribution(y_train, \"Train\")\nval_dist = analyze_split_distribution(y_val, \"Validation\")\ntest_dist = analyze_split_distribution(y_test, \"Test\")

# 5. Visualize split distributions
print(f\"\n5️⃣ Visualizing Split Distributions:\")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Label distribution comparison
split_names = ['Train', 'Validation', 'Test']
split_dists = [train_dist, val_dist, test_dist]

# Plot 1: Total samples per split
sample_counts = [len(y_train), len(y_val), len(y_test)]
axes[0, 0].bar(split_names, sample_counts, color=['blue', 'orange', 'green'])
axes[0, 0].set_title('Sample Count per Split', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Number of Samples')
for i, v in enumerate(sample_counts):
    axes[0, 0].text(i, v + 10, str(v), ha='center', fontweight='bold')

# Plot 2: Total labels per split
total_labels = [dist.sum() for dist in split_dists]
axes[0, 1].bar(split_names, total_labels, color=['blue', 'orange', 'green'])
axes[0, 1].set_title('Total Labels per Split', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Number of Labels')
for i, v in enumerate(total_labels):
    axes[0, 1].text(i, v + 50, str(v), ha='center', fontweight='bold')

# Plot 3: Average labels per sample
avg_labels = [total_labels[i] / sample_counts[i] for i in range(3)]
axes[1, 0].bar(split_names, avg_labels, color=['blue', 'orange', 'green'])
axes[1, 0].set_title('Average Labels per Sample', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Average Labels')
for i, v in enumerate(avg_labels):
    axes[1, 0].text(i, v + 0.01, f'{v:.2f}', ha='center', fontweight='bold')

# Plot 4: Label distribution across SDGs (stacked)
x_pos = np.arange(len(mlb.classes_))
width = 0.25

axes[1, 1].bar(x_pos - width, train_dist, width, label='Train', alpha=0.8)
axes[1, 1].bar(x_pos, val_dist, width, label='Validation', alpha=0.8)
axes[1, 1].bar(x_pos + width, test_dist, width, label='Test', alpha=0.8)

axes[1, 1].set_title('SDG Label Distribution Across Splits', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('SDG Classes')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels(mlb.classes_, rotation=45, ha='right')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# 6. Data summary
print(f\"\n📋 DATA SPLIT SUMMARY:\")
print(f\"   Original dataset: {len(df_processed):,} samples\")
print(f\"   Training set: {len(y_train):,} samples ({len(y_train)/len(df_processed)*100:.1f}%)\")
print(f\"   Validation set: {len(y_val):,} samples ({len(y_val)/len(df_processed)*100:.1f}%)\")
print(f\"   Test set: {len(y_test):,} samples ({len(y_test)/len(df_processed)*100:.1f}%)\")
print(f\"   Number of features: {X_train.shape[1]:,}\")
print(f\"   Number of enhanced features: {X_enhanced_train.shape[1]:,}\")
print(f\"   Number of target classes: {y_train.shape[1]}\")

## 7. Multi-label Model Training

### One-vs-Rest & Hybrid Ensemble Approaches

In [ ]:
# Multi-label Model Training
print("🤖 MULTI-LABEL MODEL TRAINING")
print("="*50)

# 1. Define base models
print("\n1️⃣ Defining Base Models:")

base_models = {
    'logistic_regression': MultiOutputClassifier(
        LogisticRegression(random_state=42, max_iter=1000)
    ),
    'random_forest': MultiOutputClassifier(
        RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    )
}

# Add XGBoost if available
try:
    import xgboost as xgb
    base_models['xgboost'] = MultiOutputClassifier(
        xgb.XGBClassifier(random_state=42, eval_metric='logloss')
    )
    print("   ✅ XGBoost included")
except ImportError:
    print("   ⚠️ XGBoost not available, skipping")

print(f"   Available models: {list(base_models.keys())}")

# 2. Train base models
print("\n2️⃣ Training Base Models:")

trained_models = {}
model_scores = {}

for name, model in base_models.items():
    print(f"   Training {name}...")
    
    # Train on original features
    start_time = pd.Timestamp.now()
    model.fit(X_train, y_train)
    training_time = (pd.Timestamp.now() - start_time).total_seconds()
    
    # Evaluate on validation set
    y_val_pred = model.predict(X_val)
    
    # Calculate metrics
    exact_match = accuracy_score(y_val, y_val_pred)
    hamming = hamming_loss(y_val, y_val_pred)
    f1_micro = f1_score(y_val, y_val_pred, average='micro')
    f1_macro = f1_score(y_val, y_val_pred, average='macro')
    jaccard = jaccard_score(y_val, y_val_pred, average='samples')
    
    trained_models[name] = model
    model_scores[name] = {
        'exact_match': exact_match,
        'hamming_loss': hamming,
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'jaccard': jaccard,
        'training_time': training_time
    }
    
    print(f"      ✅ Exact Match: {exact_match:.4f}")
    print(f"      ✅ F1-Micro: {f1_micro:.4f}")
    print(f"      ✅ Training time: {training_time:.1f}s")

# 3. Train ensemble models
print("\n3️⃣ Training Ensemble Models:")

# Voting Classifier
print("   Training Voting Classifier...")
voting_clf = VotingClassifier(
    estimators=[(name, model) for name, model in base_models.items()],
    voting='hard'  # Use hard voting for multi-output
)

start_time = pd.Timestamp.now()
voting_clf.fit(X_train, y_train)
voting_time = (pd.Timestamp.now() - start_time).total_seconds()

y_val_pred_voting = voting_clf.predict(X_val)

# Evaluate voting classifier
voting_scores = {
    'exact_match': accuracy_score(y_val, y_val_pred_voting),
    'hamming_loss': hamming_loss(y_val, y_val_pred_voting),
    'f1_micro': f1_score(y_val, y_val_pred_voting, average='micro'),
    'f1_macro': f1_score(y_val, y_val_pred_voting, average='macro'),
    'jaccard': jaccard_score(y_val, y_val_pred_voting, average='samples'),
    'training_time': voting_time
}

trained_models['voting_classifier'] = voting_clf
model_scores['voting_classifier'] = voting_scores

print(f"      ✅ Voting - F1-Micro: {voting_scores['f1_micro']:.4f}")

# 4. Train hybrid models (with cluster features)
print("\n4️⃣ Training Hybrid Models (with Cluster Features):")

hybrid_models = {}
hybrid_scores = {}

for name, base_model in base_models.items():
    hybrid_name = f"hybrid_{name}"
    print(f"   Training {hybrid_name}...")
    
    # Create new instance for hybrid training
    if name == 'logistic_regression':
        hybrid_model = MultiOutputClassifier(
            LogisticRegression(random_state=42, max_iter=1000)
        )
    elif name == 'random_forest':
        hybrid_model = MultiOutputClassifier(
            RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
        )
    elif name == 'xgboost':
        hybrid_model = MultiOutputClassifier(
            xgb.XGBClassifier(random_state=42, eval_metric='logloss')
        )
    
    # Train on enhanced features (with cluster information)
    start_time = pd.Timestamp.now()
    hybrid_model.fit(X_enhanced_train, y_train)
    hybrid_time = (pd.Timestamp.now() - start_time).total_seconds()
    
    # Evaluate on validation set
    y_val_pred_hybrid = hybrid_model.predict(X_enhanced_val)
    
    # Calculate metrics
    hybrid_score = {
        'exact_match': accuracy_score(y_val, y_val_pred_hybrid),
        'hamming_loss': hamming_loss(y_val, y_val_pred_hybrid),
        'f1_micro': f1_score(y_val, y_val_pred_hybrid, average='micro'),
        'f1_macro': f1_score(y_val, y_val_pred_hybrid, average='macro'),
        'jaccard': jaccard_score(y_val, y_val_pred_hybrid, average='samples'),
        'training_time': hybrid_time
    }
    
    hybrid_models[hybrid_name] = hybrid_model
    hybrid_scores[hybrid_name] = hybrid_score
    
    print(f"      ✅ {hybrid_name} - F1-Micro: {hybrid_score['f1_micro']:.4f}")

# Combine all models and scores
all_models = {**trained_models, **hybrid_models}
all_scores = {**model_scores, **hybrid_scores}

# 5. Model comparison
print("\n5️⃣ Model Performance Comparison:")

# Create comparison DataFrame
comparison_df = pd.DataFrame(all_scores).T
comparison_df = comparison_df.round(4)
comparison_df = comparison_df.sort_values('f1_micro', ascending=False)

print(f"\n📊 Model Performance Summary (sorted by F1-Micro):")
print(comparison_df)

# Find best model
best_model_name = comparison_df.index[0]
best_score = comparison_df.loc[best_model_name, 'f1_micro']

print(f"\n🏆 Best Model: {best_model_name} (F1-Micro: {best_score:.4f})")

# 6. Visualize model comparison
print("\n6️⃣ Visualizing Model Performance:")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: F1-Score comparison
model_names = list(all_scores.keys())
f1_micro_scores = [all_scores[name]['f1_micro'] for name in model_names]
f1_macro_scores = [all_scores[name]['f1_macro'] for name in model_names]

x_pos = np.arange(len(model_names))
width = 0.35

axes[0, 0].bar(x_pos - width/2, f1_micro_scores, width, label='F1-Micro', alpha=0.8)
axes[0, 0].bar(x_pos + width/2, f1_macro_scores, width, label='F1-Macro', alpha=0.8)
axes[0, 0].set_title('F1-Score Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Models')
axes[0, 0].set_ylabel('F1-Score')
axes[0, 0].set_xticks(x_pos)
axes[0, 0].set_xticklabels(model_names, rotation=45, ha='right')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Exact Match vs Jaccard
exact_match_scores = [all_scores[name]['exact_match'] for name in model_names]
jaccard_scores = [all_scores[name]['jaccard'] for name in model_names]

axes[0, 1].bar(x_pos - width/2, exact_match_scores, width, label='Exact Match', alpha=0.8)
axes[0, 1].bar(x_pos + width/2, jaccard_scores, width, label='Jaccard', alpha=0.8)
axes[0, 1].set_title('Exact Match vs Jaccard Similarity', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Models')
axes[0, 1].set_ylabel('Score')
axes[0, 1].set_xticks(x_pos)
axes[0, 1].set_xticklabels(model_names, rotation=45, ha='right')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Hamming Loss
hamming_scores = [all_scores[name]['hamming_loss'] for name in model_names]

axes[1, 0].bar(model_names, hamming_scores, alpha=0.8, color='red')
axes[1, 0].set_title('Hamming Loss (Lower is Better)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Models')
axes[1, 0].set_ylabel('Hamming Loss')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Training Time
training_times = [all_scores[name]['training_time'] for name in model_names]

axes[1, 1].bar(model_names, training_times, alpha=0.8, color='green')
axes[1, 1].set_title('Training Time (seconds)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Models')
axes[1, 1].set_ylabel('Time (seconds)')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ Model training completed!")
print(f"   Total models trained: {len(all_models)}")
print(f"   Best performing model: {best_model_name}")

## 8. Model Evaluation & Visualization

### Comprehensive Multi-label Metrics and Confusion Matrix Analysis

In [ ]:
# Comprehensive Model Evaluation
print("📈 COMPREHENSIVE MODEL EVALUATION")
print("="*50)

# 1. Evaluate best model on test set
print(f"\n1️⃣ Final Evaluation of Best Model ({best_model_name}):")

best_model = all_models[best_model_name]

# Determine if it's a hybrid model
if best_model_name.startswith('hybrid_'):
    y_test_pred = best_model.predict(X_enhanced_test)
    X_test_eval = X_enhanced_test
else:
    y_test_pred = best_model.predict(X_test)
    X_test_eval = X_test

# Calculate comprehensive metrics
test_metrics = {
    'exact_match_ratio': accuracy_score(y_test, y_test_pred),
    'hamming_loss': hamming_loss(y_test, y_test_pred),
    'jaccard_similarity': jaccard_score(y_test, y_test_pred, average='samples'),
    'precision_micro': precision_score(y_test, y_test_pred, average='micro'),
    'precision_macro': precision_score(y_test, y_test_pred, average='macro'),
    'precision_weighted': precision_score(y_test, y_test_pred, average='weighted'),
    'recall_micro': recall_score(y_test, y_test_pred, average='micro'),
    'recall_macro': recall_score(y_test, y_test_pred, average='macro'),
    'recall_weighted': recall_score(y_test, y_test_pred, average='weighted'),
    'f1_micro': f1_score(y_test, y_test_pred, average='micro'),
    'f1_macro': f1_score(y_test, y_test_pred, average='macro'),
    'f1_weighted': f1_score(y_test, y_test_pred, average='weighted')
}

print(f"\n📊 Final Test Set Results:")
print(f"   Exact Match Ratio: {test_metrics['exact_match_ratio']:.4f}")
print(f"   Hamming Loss: {test_metrics['hamming_loss']:.4f}")
print(f"   Jaccard Similarity: {test_metrics['jaccard_similarity']:.4f}")
print(f"\n   Precision (Micro/Macro/Weighted): {test_metrics['precision_micro']:.4f} / {test_metrics['precision_macro']:.4f} / {test_metrics['precision_weighted']:.4f}")
print(f"   Recall (Micro/Macro/Weighted): {test_metrics['recall_micro']:.4f} / {test_metrics['recall_macro']:.4f} / {test_metrics['recall_weighted']:.4f}")
print(f"   F1-Score (Micro/Macro/Weighted): {test_metrics['f1_micro']:.4f} / {test_metrics['f1_macro']:.4f} / {test_metrics['f1_weighted']:.4f}")

# 2. Per-class performance analysis
print(f"\n2️⃣ Per-Class Performance Analysis:")

# Calculate per-class metrics
precision_per_class = precision_score(y_test, y_test_pred, average=None)
recall_per_class = recall_score(y_test, y_test_pred, average=None)
f1_per_class = f1_score(y_test, y_test_pred, average=None)

# Create per-class results DataFrame
per_class_df = pd.DataFrame({
    'SDG': mlb.classes_,
    'Precision': precision_per_class,
    'Recall': recall_per_class,
    'F1-Score': f1_per_class,
    'Support': y_test.sum(axis=0)
}).sort_values('F1-Score', ascending=False)

print(f"\n📊 Per-Class Performance (Top 10):")
print(per_class_df.head(10).round(4))

# 3. Confusion Matrix Analysis
print(f"\n3️⃣ Multi-label Confusion Matrix Analysis:")

# Calculate confusion matrix for each class
cm_array = multilabel_confusion_matrix(y_test, y_test_pred)

# Display confusion matrix for top 3 performing classes
top_3_classes = per_class_df.head(3)['SDG'].values
top_3_indices = [list(mlb.classes_).index(sdg) for sdg in top_3_classes]

print(f"   Showing confusion matrices for top 3 performing classes:")
for i, (sdg, idx) in enumerate(zip(top_3_classes, top_3_indices)):
    cm = cm_array[idx]
    tn, fp, fn, tp = cm.ravel()
    print(f"\n   {sdg}:")
    print(f"      True Negatives: {tn}, False Positives: {fp}")
    print(f"      False Negatives: {fn}, True Positives: {tp}")
    print(f"      Precision: {tp/(tp+fp):.4f}, Recall: {tp/(tp+fn):.4f}")

# 4. Visualizations
print(f"\n4️⃣ Creating Detailed Visualizations:")

# Visualization 1: Per-class performance
fig, axes = plt.subplots(2, 2, figsize=(20, 16))

# Plot 1: Per-class F1-scores
axes[0, 0].bar(range(len(per_class_df)), per_class_df['F1-Score'], alpha=0.8)
axes[0, 0].set_title('Per-Class F1-Score Performance', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('SDG Classes')
axes[0, 0].set_ylabel('F1-Score')
axes[0, 0].set_xticks(range(len(per_class_df)))
axes[0, 0].set_xticklabels(per_class_df['SDG'], rotation=45, ha='right')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Precision vs Recall scatter
axes[0, 1].scatter(per_class_df['Precision'], per_class_df['Recall'], 
                  s=per_class_df['Support']*2, alpha=0.6)
axes[0, 1].set_title('Precision vs Recall (bubble size = support)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Precision')
axes[0, 1].set_ylabel('Recall')
axes[0, 1].grid(True, alpha=0.3)

# Add labels for each point
for i, row in per_class_df.iterrows():
    axes[0, 1].annotate(row['SDG'], (row['Precision'], row['Recall']), 
                       xytext=(5, 5), textcoords='offset points', fontsize=8)

# Plot 3: Metrics comparison
metrics_names = ['Precision', 'Recall', 'F1-Score']
micro_scores = [test_metrics['precision_micro'], test_metrics['recall_micro'], test_metrics['f1_micro']]
macro_scores = [test_metrics['precision_macro'], test_metrics['recall_macro'], test_metrics['f1_macro']]
weighted_scores = [test_metrics['precision_weighted'], test_metrics['recall_weighted'], test_metrics['f1_weighted']]

x_pos = np.arange(len(metrics_names))
width = 0.25

axes[1, 0].bar(x_pos - width, micro_scores, width, label='Micro', alpha=0.8)
axes[1, 0].bar(x_pos, macro_scores, width, label='Macro', alpha=0.8)
axes[1, 0].bar(x_pos + width, weighted_scores, width, label='Weighted', alpha=0.8)

axes[1, 0].set_title('Averaging Methods Comparison', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Metrics')
axes[1, 0].set_ylabel('Score')
axes[1, 0].set_xticks(x_pos)
axes[1, 0].set_xticklabels(metrics_names)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Model comparison (validation vs test)
if len(all_scores) > 1:
    # Get top 5 models for comparison
    top_models = list(comparison_df.head(5).index)
    
    val_f1_scores = [all_scores[model]['f1_micro'] for model in top_models]
    
    # Calculate test scores for top models
    test_f1_scores = []
    for model_name in top_models:
        model = all_models[model_name]
        if model_name.startswith('hybrid_'):
            y_pred_temp = model.predict(X_enhanced_test)
        else:
            y_pred_temp = model.predict(X_test)
        test_f1_scores.append(f1_score(y_test, y_pred_temp, average='micro'))
    
    x_pos = np.arange(len(top_models))
    width = 0.35
    
    axes[1, 1].bar(x_pos - width/2, val_f1_scores, width, label='Validation', alpha=0.8)
    axes[1, 1].bar(x_pos + width/2, test_f1_scores, width, label='Test', alpha=0.8)
    axes[1, 1].set_title('Validation vs Test Performance', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Models')
    axes[1, 1].set_ylabel('F1-Score (Micro)')
    axes[1, 1].set_xticks(x_pos)
    axes[1, 1].set_xticklabels(top_models, rotation=45, ha='right')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 5. Confusion Matrix Heatmap for top classes
print(f"\n5️⃣ Confusion Matrix Heatmaps:")

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (sdg, idx) in enumerate(zip(top_3_classes, top_3_indices)):
    cm = cm_array[idx]
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
    axes[i].set_title(f'Confusion Matrix - {sdg}', fontsize=12, fontweight='bold')
    axes[i].set_xlabel('Predicted')
    axes[i].set_ylabel('Actual')

plt.tight_layout()
plt.show()

# 6. Error Analysis
print(f"\n6️⃣ Error Analysis:")

# Calculate prediction errors
prediction_errors = np.abs(y_test - y_test_pred)
error_per_sample = prediction_errors.sum(axis=1)
error_per_class = prediction_errors.sum(axis=0)

print(f"   Total prediction errors: {prediction_errors.sum():,}")
print(f"   Average errors per sample: {error_per_sample.mean():.2f}")
print(f"   Samples with no errors: {sum(error_per_sample == 0):,} ({sum(error_per_sample == 0)/len(error_per_sample)*100:.1f}%)")
print(f"   Samples with 1 error: {sum(error_per_sample == 1):,} ({sum(error_per_sample == 1)/len(error_per_sample)*100:.1f}%)")
print(f"   Samples with 2+ errors: {sum(error_per_sample >= 2):,} ({sum(error_per_sample >= 2)/len(error_per_sample)*100:.1f}%)")

# Most problematic classes
error_class_df = pd.DataFrame({
    'SDG': mlb.classes_,
    'Total_Errors': error_per_class,
    'Error_Rate': error_per_class / y_test.sum(axis=0)
}).sort_values('Total_Errors', ascending=False)

print(f"\n   Most problematic classes (by total errors):")
print(error_class_df.head().round(4))

print(f"\n✅ Comprehensive evaluation completed!")
print(f"   Best model: {best_model_name}")
print(f"   Final F1-Score (Micro): {test_metrics['f1_micro']:.4f}")
print(f"   Final F1-Score (Macro): {test_metrics['f1_macro']:.4f}")
print(f"   Exact Match Ratio: {test_metrics['exact_match_ratio']:.4f}")

## 9. Hyperparameter Tuning & Cross-Validation

### GridSearchCV and Stratified K-Fold for Multi-label Optimization

In [ ]:
# Hyperparameter Tuning and Cross-Validation
print("🔧 HYPERPARAMETER TUNING & CROSS-VALIDATION")
print("="*60)

# 1. Define parameter grids for tuning
print("\n1️⃣ Defining Parameter Grids:")

param_grids = {
    'logistic_regression': {
        'estimator__C': [0.1, 1.0, 10.0],
        'estimator__penalty': ['l1', 'l2'],
        'estimator__solver': ['liblinear']
    },
    'random_forest': {
        'estimator__n_estimators': [50, 100, 200],
        'estimator__max_depth': [10, 20, None],
        'estimator__min_samples_split': [2, 5, 10]
    }
}

# Add XGBoost parameters if available
if 'xgboost' in base_models:
    param_grids['xgboost'] = {
        'estimator__n_estimators': [50, 100, 200],
        'estimator__max_depth': [3, 6, 10],
        'estimator__learning_rate': [0.01, 0.1, 0.2]
    }

print(f"   Parameter grids defined for: {list(param_grids.keys())}")

# 2. Cross-validation setup
print("\n2️⃣ Setting up Cross-Validation:")

from sklearn.metrics import make_scorer

# Define scoring metric
scorer = make_scorer(f1_score, average='micro')

# Combine training and validation sets for cross-validation
X_train_full = np.vstack([X_train.toarray(), X_val.toarray()])
X_enhanced_train_full = np.vstack([X_enhanced_train.toarray(), X_enhanced_val.toarray()])
y_train_full = np.vstack([y_train, y_val])

print(f"   Full training set shape: {X_train_full.shape}")
print(f"   Enhanced training set shape: {X_enhanced_train_full.shape}")
print(f"   Scoring metric: F1-Score (Micro)")

# 3. Hyperparameter tuning for selected models
print("\n3️⃣ Performing Hyperparameter Tuning:")

tuned_models = {}
tuning_results = {}

# Tune top 2 models to save computation time
models_to_tune = ['logistic_regression', 'random_forest']

for model_name in models_to_tune:
    if model_name in param_grids:
        print(f"\n   Tuning {model_name}...")
        
        # Create fresh model instance
        if model_name == 'logistic_regression':
            base_model = MultiOutputClassifier(
                LogisticRegression(random_state=42, max_iter=1000)
            )
        elif model_name == 'random_forest':
            base_model = MultiOutputClassifier(
                RandomForestClassifier(random_state=42, n_jobs=-1)
            )
        
        # Perform grid search
        grid_search = GridSearchCV(
            base_model, 
            param_grids[model_name], 
            cv=3,  # 3-fold CV to save time
            scoring=scorer, 
            n_jobs=-1, 
            verbose=1
        )
        
        # Fit grid search
        start_time = pd.Timestamp.now()
        grid_search.fit(X_train_full, y_train_full)
        tuning_time = (pd.Timestamp.now() - start_time).total_seconds()
        
        # Store results
        tuned_models[f"tuned_{model_name}"] = grid_search.best_estimator_
        tuning_results[model_name] = {
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'tuning_time': tuning_time
        }
        
        print(f"      ✅ Best score: {grid_search.best_score_:.4f}")
        print(f"      ✅ Best params: {grid_search.best_params_}")
        print(f"      ✅ Tuning time: {tuning_time:.1f}s")

# 4. Cross-validation analysis
print("\n4️⃣ Cross-Validation Analysis:")

cv_results = {}

# Perform cross-validation on the best models
top_models_for_cv = ['logistic_regression', 'random_forest']
if 'xgboost' in base_models:
    top_models_for_cv.append('xgboost')

for model_name in top_models_for_cv:
    print(f"\n   Cross-validating {model_name}...")
    
    # Use tuned model if available, otherwise use base model
    if f"tuned_{model_name}" in tuned_models:
        model = tuned_models[f"tuned_{model_name}"]
        print(f"      Using tuned parameters")
    else:
        model = base_models[model_name]
        print(f"      Using default parameters")
    
    # Perform cross-validation
    cv_scores = cross_val_score(model, X_train_full, y_train_full, 
                               cv=5, scoring=scorer, n_jobs=-1)
    
    cv_results[model_name] = {
        'scores': cv_scores,
        'mean': cv_scores.mean(),
        'std': cv_scores.std()
    }
    
    print(f"      CV Scores: {cv_scores}")
    print(f"      Mean ± Std: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# 5. Hybrid model tuning
print("\n5️⃣ Hybrid Model Optimization:")

# Test if adding cluster features improves the best model
best_base_model = max(cv_results.items(), key=lambda x: x[1]['mean'])
best_base_name = best_base_model[0]

print(f"   Best base model: {best_base_name}")
print(f"   Testing hybrid version with cluster features...")

# Create hybrid version of best model
if best_base_name == 'logistic_regression':
    hybrid_model = MultiOutputClassifier(
        LogisticRegression(random_state=42, max_iter=1000)
    )
elif best_base_name == 'random_forest':
    hybrid_model = MultiOutputClassifier(
        RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    )

# Cross-validate hybrid model
hybrid_cv_scores = cross_val_score(hybrid_model, X_enhanced_train_full, y_train_full, 
                                  cv=5, scoring=scorer, n_jobs=-1)

print(f"   Hybrid CV Scores: {hybrid_cv_scores}")
print(f"   Hybrid Mean ± Std: {hybrid_cv_scores.mean():.4f} ± {hybrid_cv_scores.std():.4f}")

# Compare base vs hybrid
improvement = hybrid_cv_scores.mean() - cv_results[best_base_name]['mean']
print(f"   Improvement with clustering: {improvement:.4f}")

if improvement > 0:
    print(f"   ✅ Hybrid approach shows improvement!")
else:
    print(f"   ⚠️ Hybrid approach shows minimal improvement")

# 6. Results visualization
print("\n6️⃣ Visualizing Tuning Results:")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Cross-validation scores comparison
model_names = list(cv_results.keys())
cv_means = [cv_results[name]['mean'] for name in model_names]
cv_stds = [cv_results[name]['std'] for name in model_names]

axes[0, 0].bar(model_names, cv_means, yerr=cv_stds, capsize=5, alpha=0.8)
axes[0, 0].set_title('Cross-Validation Scores Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Models')
axes[0, 0].set_ylabel('F1-Score (Mean ± Std)')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Base vs Hybrid comparison
if improvement is not None:
    comparison_models = [best_base_name, f'hybrid_{best_base_name}']
    comparison_scores = [cv_results[best_base_name]['mean'], hybrid_cv_scores.mean()]
    comparison_stds = [cv_results[best_base_name]['std'], hybrid_cv_scores.std()]
    
    axes[0, 1].bar(comparison_models, comparison_scores, yerr=comparison_stds, 
                   capsize=5, alpha=0.8, color=['blue', 'orange'])
    axes[0, 1].set_title('Base vs Hybrid Model Comparison', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Model Type')
    axes[0, 1].set_ylabel('F1-Score (Mean ± Std)')
    axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Hyperparameter tuning results
if tuning_results:
    tuned_model_names = list(tuning_results.keys())
    tuning_scores = [tuning_results[name]['best_score'] for name in tuned_model_names]
    base_scores = [cv_results[name]['mean'] for name in tuned_model_names]
    
    x_pos = np.arange(len(tuned_model_names))
    width = 0.35
    
    axes[1, 0].bar(x_pos - width/2, base_scores, width, label='Default', alpha=0.8)
    axes[1, 0].bar(x_pos + width/2, tuning_scores, width, label='Tuned', alpha=0.8)
    axes[1, 0].set_title('Default vs Tuned Parameters', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Models')
    axes[1, 0].set_ylabel('F1-Score')
    axes[1, 0].set_xticks(x_pos)
    axes[1, 0].set_xticklabels(tuned_model_names)
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

# Plot 4: CV score distributions
if cv_results:
    # Create box plot of CV scores
    cv_scores_list = [cv_results[name]['scores'] for name in model_names]
    
    axes[1, 1].boxplot(cv_scores_list, labels=model_names)
    axes[1, 1].set_title('Cross-Validation Score Distributions', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Models')
    axes[1, 1].set_ylabel('F1-Score')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 7. Final model selection
print("\n7️⃣ Final Model Selection:")

final_model_candidates = {}

# Add cross-validated models
for name, results in cv_results.items():
    final_model_candidates[name] = results['mean']

# Add hybrid model
if improvement is not None:
    final_model_candidates[f'hybrid_{best_base_name}'] = hybrid_cv_scores.mean()

# Add tuned models
for name, results in tuning_results.items():
    final_model_candidates[f'tuned_{name}'] = results['best_score']

# Find the best final model
final_best_model = max(final_model_candidates.items(), key=lambda x: x[1])
final_best_name, final_best_score = final_best_model

print(f"\n🏆 FINAL MODEL SELECTION:")
print(f"   Best Model: {final_best_name}")
print(f"   Best Score: {final_best_score:.4f}")

# Display all candidates
print(f"\n📊 All Final Candidates:")
sorted_candidates = sorted(final_model_candidates.items(), key=lambda x: x[1], reverse=True)
for name, score in sorted_candidates:
    print(f"   {name:25s}: {score:.4f}")

print(f"\n✅ Hyperparameter tuning and cross-validation completed!")
print(f"   Best approach: {final_best_name}")
print(f"   Expected performance: {final_best_score:.4f}")

## 10. Model Saving & Inference Example

### Save Models, Vectorizer, and Pipeline for Production Use

In [ ]:
# Model Saving and Inference Pipeline
print("💾 MODEL SAVING & INFERENCE PIPELINE")
print("="*50)

# 1. Prepare models for saving
print("\n1️⃣ Preparing Models for Saving:")

# Create models directory
import os
models_dir = "../models"
os.makedirs(models_dir, exist_ok=True)

# Determine the final best model to save
if final_best_name.startswith('tuned_'):
    model_to_save = tuned_models[final_best_name]
    print(f"   Selected tuned model: {final_best_name}")
elif final_best_name.startswith('hybrid_'):
    # Train the hybrid model one more time on full training data
    base_name = final_best_name.replace('hybrid_', '')
    if base_name == 'logistic_regression':
        model_to_save = MultiOutputClassifier(
            LogisticRegression(random_state=42, max_iter=1000)
        )
    elif base_name == 'random_forest':
        model_to_save = MultiOutputClassifier(
            RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
        )
    
    model_to_save.fit(X_enhanced_train_full, y_train_full)
    print(f"   Selected hybrid model: {final_best_name}")
else:
    model_to_save = all_models[final_best_name]
    print(f"   Selected base model: {final_best_name}")

print(f"   Model type: {type(model_to_save)}")

# 2. Save all components
print("\n2️⃣ Saving Model Components:")

try:
    # Save the main model
    model_path = os.path.join(models_dir, "best_model.joblib")
    joblib.dump(model_to_save, model_path)
    print(f"   ✅ Model saved: {model_path}")
    
    # Save the TF-IDF vectorizer
    vectorizer_path = os.path.join(models_dir, "tfidf_vectorizer.joblib")
    joblib.dump(tfidf, vectorizer_path)
    print(f"   ✅ TF-IDF vectorizer saved: {vectorizer_path}")
    
    # Save the feature scaler
    scaler_path = os.path.join(models_dir, "feature_scaler.joblib")
    joblib.dump(scaler, scaler_path)
    print(f"   ✅ Feature scaler saved: {scaler_path}")
    
    # Save the multi-label binarizer
    mlb_path = os.path.join(models_dir, "multilabel_binarizer.joblib")
    joblib.dump(mlb, mlb_path)
    print(f"   ✅ Multi-label binarizer saved: {mlb_path}")
    
    # Save the K-means clustering model
    kmeans_path = os.path.join(models_dir, "kmeans_model.joblib")
    joblib.dump(kmeans_final, kmeans_path)
    print(f"   ✅ K-means model saved: {kmeans_path}")
    
    # Save model metadata
    metadata = {
        'model_name': final_best_name,
        'model_score': final_best_score,
        'feature_names': tfidf.get_feature_names_out().tolist() + ['Year', 'Cited'],
        'target_names': mlb.classes_.tolist(),
        'n_clusters': best_k_silhouette,
        'is_hybrid': final_best_name.startswith('hybrid_'),
        'test_metrics': test_metrics
    }
    
    metadata_path = os.path.join(models_dir, "model_metadata.joblib")
    joblib.dump(metadata, metadata_path)
    print(f"   ✅ Model metadata saved: {metadata_path}")
    
except Exception as e:
    print(f"   ❌ Error saving models: {e}")

# 3. Create inference pipeline
print("\n3️⃣ Creating Inference Pipeline:")

class SDGPredictor:
    \"\"\"Production-ready SDG prediction pipeline\"\"\"
    
    def __init__(self, models_dir):
        self.models_dir = models_dir
        self.model = None
        self.tfidf = None
        self.scaler = None
        self.mlb = None
        self.kmeans = None
        self.metadata = None
        self.is_loaded = False
        
    def load_models(self):
        \"\"\"Load all saved models and components\"\"\"
        try:
            self.model = joblib.load(os.path.join(self.models_dir, "best_model.joblib"))
            self.tfidf = joblib.load(os.path.join(self.models_dir, "tfidf_vectorizer.joblib"))
            self.scaler = joblib.load(os.path.join(self.models_dir, "feature_scaler.joblib"))
            self.mlb = joblib.load(os.path.join(self.models_dir, "multilabel_binarizer.joblib"))
            self.kmeans = joblib.load(os.path.join(self.models_dir, "kmeans_model.joblib"))
            self.metadata = joblib.load(os.path.join(self.models_dir, "model_metadata.joblib"))
            
            self.is_loaded = True
            return True
        except Exception as e:
            print(f"Error loading models: {e}")
            return False
    
    def preprocess_text(self, text):
        \"\"\"Clean and preprocess text\"\"\"
        if pd.isna(text):
            return ""
        
        text = str(text).lower()
        text = re.sub(r'[^a-zA-Z\\s]', '', text)
        text = ' '.join(text.split())
        return text
    
    def predict(self, title, year=None, cited=None):
        \"\"\"Predict SDG labels for a single publication\"\"\"
        if not self.is_loaded:
            raise ValueError("Models not loaded. Call load_models() first.")
        
        # Preprocess inputs
        title_clean = self.preprocess_text(title)
        year = year if year else 2020
        cited = cited if cited else 0
        
        # Create feature vector
        tfidf_features = self.tfidf.transform([title_clean])
        numerical_features = self.scaler.transform([[year, cited]])
        
        # Combine features
        from scipy.sparse import hstack, csr_matrix
        combined_features = hstack([tfidf_features, csr_matrix(numerical_features)])
        
        # Add cluster feature if hybrid model
        if self.metadata['is_hybrid']:
            cluster_label = self.kmeans.predict(combined_features)[0]
            enhanced_features = hstack([combined_features, csr_matrix([[cluster_label]])])
            features_final = enhanced_features
        else:
            features_final = combined_features
        
        # Make prediction
        y_pred = self.model.predict(features_final)
        y_proba = None
        
        # Get probabilities if available
        if hasattr(self.model, 'predict_proba'):
            try:
                y_proba = self.model.predict_proba(features_final)
            except:
                pass
        
        # Convert to readable format
        predicted_sdgs = []
        probabilities = {}
        
        for i, class_name in enumerate(self.metadata['target_names']):
            if y_pred[0][i] == 1:
                predicted_sdgs.append(class_name)
            
            if y_proba is not None:
                if isinstance(y_proba, list):
                    probabilities[class_name] = float(y_proba[i][0][1])
                else:
                    probabilities[class_name] = float(y_proba[0][i])
        
        return {
            'title': title,
            'predicted_sdgs': predicted_sdgs,
            'probabilities': probabilities,
            'model_used': self.metadata['model_name']
        }

# Initialize and test the predictor
predictor = SDGPredictor(models_dir)
if predictor.load_models():
    print(f"   ✅ Inference pipeline loaded successfully!")
    print(f"   Model: {predictor.metadata['model_name']}")
    print(f"   Expected score: {predictor.metadata['model_score']:.4f}")
else:
    print(f"   ❌ Failed to load inference pipeline")

# 4. Test inference with examples
print("\n4️⃣ Testing Inference Pipeline:")

test_examples = [
    {
        'title': 'Machine learning approaches for sustainable energy management',
        'year': 2023,
        'cited': 15
    },
    {
        'title': 'Climate change impacts on biodiversity conservation',
        'year': 2022,
        'cited': 45
    },
    {
        'title': 'Healthcare innovation using artificial intelligence',
        'year': 2023,
        'cited': 28
    },
    {
        'title': 'Educational technology for inclusive learning environments',
        'year': 2021,
        'cited': 12
    }
]

if predictor.is_loaded:
    print(f"\\n   Testing with {len(test_examples)} sample publications:")
    
    for i, example in enumerate(test_examples):
        try:
            result = predictor.predict(
                title=example['title'],
                year=example['year'],
                cited=example['cited']
            )
            
            print(f"\\n   Example {i+1}:")
            print(f"      Title: {example['title'][:60]}...")
            print(f"      Predicted SDGs: {', '.join(result['predicted_sdgs']) if result['predicted_sdgs'] else 'None'}")
            
            if result['probabilities']:
                top_probs = sorted(result['probabilities'].items(), key=lambda x: x[1], reverse=True)[:3]
                print(f"      Top probabilities: {', '.join([f'{sdg}({prob:.3f})' for sdg, prob in top_probs])}")
            
        except Exception as e:
            print(f"   ❌ Error in example {i+1}: {e}")

# 5. Create standalone prediction script
print("\\n5️⃣ Creating Standalone Prediction Script:")

predict_script = '''#!/usr/bin/env python3
\"\"\"
Standalone SDG prediction script
Usage: python predict_sample.py "publication title" [year] [cited]
\"\"\"

import sys
import pandas as pd
import joblib
import os
import re
from scipy.sparse import hstack, csr_matrix

class SDGPredictor:
    def __init__(self, models_dir="models"):
        self.models_dir = models_dir
        self.model = None
        self.tfidf = None
        self.scaler = None
        self.mlb = None
        self.kmeans = None
        self.metadata = None
        self.is_loaded = False
        
    def load_models(self):
        try:
            self.model = joblib.load(os.path.join(self.models_dir, "best_model.joblib"))
            self.tfidf = joblib.load(os.path.join(self.models_dir, "tfidf_vectorizer.joblib"))
            self.scaler = joblib.load(os.path.join(self.models_dir, "feature_scaler.joblib"))
            self.mlb = joblib.load(os.path.join(self.models_dir, "multilabel_binarizer.joblib"))
            self.kmeans = joblib.load(os.path.join(self.models_dir, "kmeans_model.joblib"))
            self.metadata = joblib.load(os.path.join(self.models_dir, "model_metadata.joblib"))
            self.is_loaded = True
            return True
        except Exception as e:
            print(f"Error loading models: {e}")
            return False
    
    def preprocess_text(self, text):
        if pd.isna(text):
            return ""
        text = str(text).lower()
        text = re.sub(r'[^a-zA-Z\\\\s]', '', text)
        text = ' '.join(text.split())
        return text
    
    def predict(self, title, year=None, cited=None):
        if not self.is_loaded:
            raise ValueError("Models not loaded")
        
        title_clean = self.preprocess_text(title)
        year = year if year else 2020
        cited = cited if cited else 0
        
        tfidf_features = self.tfidf.transform([title_clean])
        numerical_features = self.scaler.transform([[year, cited]])
        combined_features = hstack([tfidf_features, csr_matrix(numerical_features)])
        
        if self.metadata['is_hybrid']:
            cluster_label = self.kmeans.predict(combined_features)[0]
            features_final = hstack([combined_features, csr_matrix([[cluster_label]])])
        else:
            features_final = combined_features
        
        y_pred = self.model.predict(features_final)
        predicted_sdgs = [self.metadata['target_names'][i] for i in range(len(y_pred[0])) if y_pred[0][i] == 1]
        
        return predicted_sdgs

def main():
    if len(sys.argv) < 2:
        print("Usage: python predict_sample.py \\"publication title\\" [year] [cited]")
        sys.exit(1)
    
    title = sys.argv[1]
    year = int(sys.argv[2]) if len(sys.argv) > 2 else None
    cited = int(sys.argv[3]) if len(sys.argv) > 3 else None
    
    predictor = SDGPredictor()
    if predictor.load_models():
        predicted_sdgs = predictor.predict(title, year, cited)
        print(f"Title: {title}")
        print(f"Predicted SDGs: {', '.join(predicted_sdgs) if predicted_sdgs else 'None'}")
    else:
        print("Failed to load models")

if __name__ == "__main__":
    main()
'''

# Save the prediction script
script_path = "../predict_sample.py"
with open(script_path, 'w') as f:
    f.write(predict_script)

print(f"   ✅ Standalone script saved: {script_path}")

# 6. Final summary
print("\\n6️⃣ Final Summary:")

print(f"\\n🎉 PROJECT COMPLETION SUMMARY:")
print(f"   Dataset processed: {len(df_processed):,} publications")
print(f"   Features created: {combined_features.shape[1]:,}")
print(f"   SDG classes: {len(mlb.classes_)}")
print(f"   Best model: {final_best_name}")
print(f"   Final performance: {final_best_score:.4f} F1-Score")
print(f"   Models saved to: {models_dir}")

print(f"\\n📁 Saved Components:")
print(f"   - best_model.joblib (trained classifier)")
print(f"   - tfidf_vectorizer.joblib (text vectorizer)")
print(f"   - feature_scaler.joblib (numerical scaler)")
print(f"   - multilabel_binarizer.joblib (label encoder)")
print(f"   - kmeans_model.joblib (clustering model)")
print(f"   - model_metadata.joblib (model information)")
print(f"   - predict_sample.py (inference script)")

print(f"\\n🚀 Ready for Production:")
print(f"   Use the SDGPredictor class for batch predictions")
print(f"   Use predict_sample.py for single predictions")
print(f"   All models are optimized and cross-validated")

print(f"\\n✅ NOTEBOOK EXECUTION COMPLETED SUCCESSFULLY!")'''